# Robotics, Vision & Control 3e: for Python
## Appendices

Copyright (c) 2021- Peter Corke

In [ ]:
try:
    from google.colab import output
    print('Running on CoLab')
    output.enable_custom_widget_manager()
    !pip install ipympl
    !pip install spatialmath-python
    COLAB = True
    SWIFT = False
except ModuleNotFoundError:
    COLAB = False
    SWIFT = False
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"
from IPython.display import HTML


# add RTB examples folder to the path
import sys
import os.path
import roboticstoolbox as rtb
sys.path.append(os.path.join(rtb.__path__[0], 'examples'))

# standard imports
import numpy as np
from scipy import linalg
%matplotlib widget
import matplotlib.pyplot as plt
import math
from math import pi
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from spatialmath import *
from spatialmath.base import *


# B Linear Algebra


## B.2 Matrices


### B.2.1 Square Matrices


This demo shows a red unit vector rotating like the minute hand of a clock.  The blue vector is its linear transformation.  Twice per revolution the vectors are parallel and the red vector on those occasions are eigenvectors.  The relative scale of the blue vector is the eigenvalue.

Change the four numbers to see what happens with a different transformation matrix, they are the four matrix elements given row-wise.

Hit the interrupt button to stop the animation.

In [ ]:
%run -m eigdemo 1 2 3 4

# C Geometry


## C.1 Euclidean Geometry


### C.1.2 Lines


#### C.1.2.2 Lines in 3D and Plücker Coordinates


In [ ]:
P = [2, 3, 4]; Q = [3, 5, 7];

In [ ]:
L = Line3.Join(P, Q)

In [ ]:
L.v.T
L.w.T

In [ ]:
L.skew()

In [ ]:
plotvol3([-5, 5]);
L.plot("b");

In [ ]:
L.point([0, 1, 2])

In [ ]:
[x, d] = L.closest_to_point([1, 2, 3])
x
d

In [ ]:
p, _ = L.intersect_plane([0, 0, 1, 0])
p

### C.1.4 Ellipses and Ellipsoids


In [ ]:
E = np.array([[1, 1], [1, 2]])

In [ ]:
plot_ellipse(E, [0, 0])

In [ ]:
e, v = np.linalg.eig(E)

the eigenvalues are

In [ ]:
e

and the eigenvectors are the columns of

In [ ]:
v

and the radii, the half-lengths of the major and minor axes are

In [ ]:
r = 1 / np.sqrt(e)

In [ ]:
plot_arrow((0, 0), v[:,0]*r[0], color="r", width=0.02);
plot_arrow((0, 0), v[:,1]*r[1], color="b", width=0.02);
plt.gca().set_aspect('equal', 'box')

and the orientation of the major axis, in degrees, is

In [ ]:
np.rad2deg(np.arctan2(v[1, 0], v[0, 0]))

### C.1.4.1 Drawing an Ellipse


In [ ]:
E = np.array([[1, 1], [1, 2]]);

In [ ]:
th = np.linspace(0, 2*pi, 50);
y = np.vstack([np.cos(th), np.sin(th)]);

In [ ]:
x = linalg.sqrtm(np.linalg.inv(E)) @ y;
plt.figure()
plt.plot(x[0, :], x[1, :], '.');

In [ ]:
plot_ellipse(E, [0, 0])

#### C.1.4.2 Fitting an Ellipse to Data


In [ ]:
plt.figure()
rng = np.random.default_rng(0);
# create 200 random points inside the ellipse
x = [];
while len(x) < 200: 
  p = rng.uniform(low=-2, high=2, size=(2,1))
  if np.linalg.norm(p.T @ E @ p) <= 1:
    x.append(p)
x = np.hstack(x);  # create 2 x 50 array
plt.plot(x[0, :], x[1, :], "k."); # plot them
# compute the moments
m00 = mpq_point(x, 0, 0);
m10 = mpq_point(x, 1, 0);
m01 = mpq_point(x, 0, 1);
xc = np.c_[m10, m01] / m00;
# compute the central second moments
x0 = x - xc.T;
u20 = mpq_point(x0, 2, 0);
u02 = mpq_point(x0, 0, 2);
u11 = mpq_point(x0, 1, 1);
# compute inertia tensor and ellipse matrix
J = np.array([[u20, u11], [u11, u02]]);
E_est = m00 / 4 * np.linalg.inv(J);

In [ ]:
print(f'E:\n {E}')
print(f'Eest:\n {E_est}')

In [ ]:
plt.plot(x[0, :], x[1, :], "k."); # plot data points
plot_ellipse(E_est, [0, 0], "r")          # plot fitted ellipse

## C.2 Homogeneous Coordinates


### C.2.1 Two Dimensions


#### C.2.1.1 Points and lines


In [ ]:
l1 = [1, -1, 0];
l2 = [1, -1, -1];

In [ ]:
plotvol2([-2, 2, -2, 2], new=True)
plot_homline(l1, "b");
plot_homline(l2, "r");

In [ ]:
np.cross(l1, l2)

# E Linearizations, Jacobians and Hessians

## E.4 Deriving Jacobians


In [ ]:
zrange = lambda xi, xv, w: np.array([
           np.linalg.norm(xi - xv[:2]) + w[0],
           np.arctan((xi[1] - xv[1]) / (xi[0] - xv[0])) -xv[2] + w[1]]);

In [ ]:
xv = np.r_[1, 2, pi/3]; xi = np.r_[10, 8]; w = np.r_[0, 0];
h0 = zrange(xi, xv, w)
d = 0.001;
J = np.column_stack([
       zrange(xi, xv + [d, 0, 0], w) - h0,
       zrange(xi, xv + [0, d, 0], w) - h0,
       zrange(xi, xv + [0, 0, d], w) - h0
                    ]) / d

In [ ]:
numjac(lambda x: zrange(xi, x, w), xv)

In [ ]:
from sympy import Matrix, MatrixSymbol, sqrt, atan, simplify, pycode
xi = MatrixSymbol("xi", 2, 1)
xv = MatrixSymbol("xv", 3, 1)
w = Matrix([0, 0]);

In [ ]:
zrange = lambda xi, xv, w: Matrix([
            sqrt((xi[0] - xv[0])**2 + (xi[1] - xv[1])**2) + w[0],
            atan((xi[1] - xv[1]) / (xi[0] - xv[0])) -xv[2] + w[1]]);
z = zrange(xi, xv, w)

In [ ]:
J = z.jacobian(xv)

In [ ]:
J.shape

# F Solving Systems of Equations


## F.1 Linear Problems


### F.1.1 Nonhomogeneous Systems


In [ ]:
A = np.array([[1, -2], [1, 1]]);
b = np.array([[8], [5]]);
x = linalg.solve(A, b)

# G Gaussian Random Variables

In [ ]:
x = np.linspace(-6, 6, 500);
plt.figure()
plt.plot(x, gauss1d(0, 1, x), "r");
plt.plot(x, gauss1d(0, 2**2, x), "b--");

In [ ]:
mu = 10; sigma = 2;
g = np.random.normal(loc=mu, scale=sigma, size=(100,))

In [ ]:
x, y = np.meshgrid(np.linspace(-5, 5, 100), np.linspace(-5, 5, 100));
P = np.diag([1, 2])**2;
g = gauss2d([0, 0], P, x, y);
ax = ax = plotvol3();
ax.plot_surface(x, y, g);
ax.contour(x, y, g, zdir="z", offset=-0.05);

In [ ]:
from scipy.stats.distributions import chi2
chi2.ppf(0.5, 2)

# H Kalman Filter


## H.2 Nonlinear Systems -- Extended Kalman Filter


In [ ]:
x = np.random.normal(5, 2, size=(1_000_000,));

In [ ]:
y = (x + 2)**2 / 4;

In [ ]:
plt.hist(y, bins=200, density=True, histtype="step");

# I Graphs


In [ ]:
import pgraph
g = pgraph.UGraph()

In [ ]:
np.random.seed(0)  # ensure repeatable results
for i in range(5):
  g.add_vertex(np.random.rand(2));

In [ ]:
g[1]

In [ ]:
g["#1"]

In [ ]:
g.add_edge(g[0], g[1]);
g.add_edge(g[0], g[2]);
g.add_edge(g[0], g[3]);
g.add_edge(g[1], g[2]);
g.add_edge(g[1], g[3]);
g.add_edge(g[3], g[4]);

In [ ]:
print(g)

In [ ]:
g.plot()

In [ ]:
g[1].adjacent()

In [ ]:
g[1].edges()

In [ ]:
g[1].edges()[0].endpoints

In [ ]:
g[1].edges()[0].cost

In [ ]:
g.closest((0.5, 0.5))

In [ ]:
path, length, _ = g.path_Astar(g[2], g[4])
path

In [ ]:
length

# J Peak Finding


## J.1 1D Signal


In [ ]:
from machinevisiontoolbox import mvtb_load_matfile, findpeaks, findpeaks2d
y = mvtb_load_matfile("data/peakfit.mat")["y"];
plt.plot(y, "-o");

In [ ]:
k = np.argmax(y)

In [ ]:
y[k]

In [ ]:
k, ypk = findpeaks(y)
k
ypk

In [ ]:
ypk[1] / ypk[0]

In [ ]:
findpeaks(y, interp=True)

In [ ]:
findpeaks(y, scale=5)

## J.2 2D Signal


In [ ]:
img = mvtb_load_matfile("data/peakfit.mat")["image"]

In [ ]:
k = np.argmax(img)

In [ ]:
img.ravel()[k]

In [ ]:
np.unravel_index(k, img.shape)

In [ ]:
xy = findpeaks2d(img)

In [ ]:
xy = findpeaks2d(img, interp=True)